In [1]:
import requests
import pandas as pd
#import json
#import numpy as np

In [2]:
from datetime import date

today = date.today()
today_date = today.strftime("%Y-%m-%d")
today_date

'2022-10-20'

--- here starts currency collection from ecb-------

In [3]:
# Building blocks for the URL
entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
resource = 'data'           # The resource for data queries is always'data'
flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
key = 'D.SEK.EUR.SP00.A'    # Defining the dimension values, explained below

# Define the parameters
parameters = {
    'startPeriod': '2022-01-01',  # Start date of the time series
    'endPeriod': today_date,     # End of the time series
    #'format':'jsondata'
}

In [4]:
# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request again, now requesting for CSV format
response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
#response = requests.get(request_url, params=parameters, headers={'Accept': 'application/vnd.sdmx.data+json;version=1.0.0-wd'})
# Response succesful? (Response code 200)
#print(response)

# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request again, now requesting for CSV format
response = requests.get(request_url, params=parameters, headers={'Accept': 'application/vnd.sdmx.data+json;version=1.0.0-wd'})
# Response succesful? (Response code 200)
response.json()


In [5]:
open('csvs/temp.csv', 'wb').write(response.content)
currency_df = pd.read_csv('csvs/temp.csv')

In [6]:
#currency_df.keys()

In [7]:
#currency_df.tail(1)

In [8]:
currency_df_filter = currency_df.filter(['TIME_PERIOD', 'OBS_VALUE', 'KEY', 'CURRENCY'], axis=1)
currency_df_filter['TIME_PERIOD'] = pd.to_datetime(currency_df_filter['TIME_PERIOD'])
currency_df_filter = currency_df_filter.set_index('TIME_PERIOD')

currency_df_filter.tail(1)

,OBS_VALUE,KEY,CURRENCY
TIME_PERIOD,,,
2022-10-19,10.9448,EXR.D.SEK.EUR.SP00.A,SEK


rate_sek = currency_df_filter.tail(1)['OBS_VALUE'].values[0]
rate_sek

In [9]:
import scr_utils as scru

scru.to_paraquet('currency_prices',currency_df_filter)

pyarrow.Table
OBS_VALUE: double
KEY: string
CURRENCY: string
TIME_PERIOD: timestamp[ns]
----
OBS_VALUE: [[10.2958,10.2808,10.2545,10.3265,10.2839,...,11.0098,11.0035,10.9893,10.906,10.9448]]
KEY: [["EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A",...,"EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A","EXR.D.SEK.EUR.SP00.A"]]
CURRENCY: [["SEK","SEK","SEK","SEK","SEK",...,"SEK","SEK","SEK","SEK","SEK"]]
TIME_PERIOD: [[2022-01-03 00:00:00.000000000,2022-01-04 00:00:00.000000000,2022-01-05 00:00:00.000000000,2022-01-06 00:00:00.000000000,2022-01-07 00:00:00.000000000,...,2022-10-13 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-10-19 00:00:00.000000000]]

In [10]:
ts=currency_df_filter

--- end of currency collection -----